In [2]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import requests
import sncosmo
import json

from utils import get_config

/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
config = get_config()

username = config['username']
date = '2018-09-27'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
minfilters = 2      # Reported detections in at least this many filters
minrefframes = 1    # Cut away alert information where this is not positive (how can it not be, but ok...)

marshal_saveprogs = 'AMPEL'   # Candidates already in this program will not be scanned. Reg exp string, eg 'AMPEL|RCF'
marshal_savid = 42
logpath = 'logs/'
slackfilepath = config['slackfilepath']

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sncandid_soup.json', 'r') as fp:
    soupids = json.load(fp)

# log. This should be saved and pushed to github. If everyone does this we should have the full scanning
# history for this run
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)



In [4]:
df_sum = pd.read_csv(os.path.join(slackfilepath, "Summary_%s.csv"%(date)))
df_phot = pd.read_csv(os.path.join(slackfilepath, "Photometry_%s.csv"%(date)))

if minrefframes>0:
    df_phot = df_phot[df_phot["nframesref"]>minrefframes]
    logger.info("Cutting down alert info to subset with nrefframes keyword")

/home/jnordin/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (162) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
INFO:jnordin:Cutting down alert info to subset with nrefframes keyword


In [5]:
logger.info("There were %s candidates reported %s"%(df_sum.shape[0],date))

INFO:jnordin:There were 337 candidates reported 2018-09-27


In [6]:
# Find already classified SNe
i_untyped = (df_sum["T2-classification"]== "None") | (df_sum["T2-classification"].isnull())
df_classified = df_sum[ ~i_untyped]
df_tmp = df_sum[ i_untyped]
logger.info("Out of these %s already have Marshal type and will be skipped."%len(df_classified))


INFO:jnordin:Out of these 82 already have Marshal type and will be skipped.


In [7]:
# Are there any remaining candidates with spectroscopic redshifts above limit
i_highz = ( (df_tmp['T2-NEDz_z']>maxz) | (df_tmp['T2-milliquas_redshift']>maxz)  | (df_tmp['T2-SDSS_spec_z']>maxz) ) 
df_classified = df_tmp[ i_highz]
df_tmp2 = df_tmp[~i_highz]
logger.info("%s have a catalog z larger than %s  and will be skipped."%(np.sum(i_highz),maxz))


INFO:jnordin:27 have a catalog z larger than 0.1  and will be skipped.


In [8]:
# Which are already saved in the marshal?
i_saved = df_tmp2["T2-programs"].str.contains(marshal_saveprogs, regex=True, na=False, flags=re.IGNORECASE)
df_saved = df_tmp2[i_saved]
df_candidates = df_tmp2[~i_saved]
logger.info("Detected %s already saved candidates and %s new"%(len(df_saved),len(df_candidates)))

INFO:jnordin:Detected 27 already saved candidates and 201 new


In [9]:
# Examine new candidates, build a list of the onces fulfilling basic criteria
sncandidates = []
for sne in df_candidates.iterrows():
    print( sne[1]["ztf_name"] )
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    if np.min(magpsf)> minpeakmag:
        msg = "%s: Skipping %s, never reached limit %s"%(date,sne[1]["ztf_name"],minpeakmag)
        logger.info(msg)
        print(msg)
        continue
    if len(magpsf)< mindet:
        msg = "%s: Skipping %s, less than %s detections"%(date,sne[1]["ztf_name"],mindet)
        logger.info(msg)
        print(msg)
        continue

    age = np.max(jd)-np.min(jd)
    if age>maxage:
        msg = "%s: Skipping %s, age %s longer than limit %s"%(date,sne[1]["ztf_name"],age,maxage)
        logger.info(msg)
        print(msg)
        continue
        
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    if len(np.unique(phot["fid"]))<minfilters:
        msg = "%s: Skipping %s, only %s filters"%(date,sne[1]["ztf_name"],len(np.unique(phot["fid"])))
        logger.info(msg)
        print(msg)
        continue
        
    sncandidates.append(sne[1]["ztf_name"])
    continue
        
    

print("Left with %s candidates to scan"%(len(sncandidates)))

INFO:jnordin:2018-09-27: Skipping ZTF18aaoiaac, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abmsngt, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abecjrr, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18aaqzhib, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abehrdw, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18aamofne, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtnfnq, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abtlrrt, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abrwvkr, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18aawrbuc, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abimrqx, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abaaeip, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abnblhi, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18aaowwgf, less than 5 detections
INFO:jnordin:2018-09-27: S

ZTF18abslxpd
ZTF18ablrndb
ZTF18aaoiaac
2018-09-27: Skipping ZTF18aaoiaac, less than 5 detections
ZTF18abtqrwa
ZTF18abgrlpv
ZTF18abmsngt
2018-09-27: Skipping ZTF18abmsngt, only 1 filters
ZTF18abecjrr
2018-09-27: Skipping ZTF18abecjrr, only 1 filters
ZTF18aaqzhib
2018-09-27: Skipping ZTF18aaqzhib, only 1 filters
ZTF18abehrdw
2018-09-27: Skipping ZTF18abehrdw, less than 5 detections
ZTF18aamofne
2018-09-27: Skipping ZTF18aamofne, never reached limit 19.5
ZTF18abtnfnq
2018-09-27: Skipping ZTF18abtnfnq, only 1 filters
ZTF18abtlrrt
2018-09-27: Skipping ZTF18abtlrrt, never reached limit 19.5
ZTF17aabihdn
ZTF18aawlhkh
ZTF18abrwvkr
2018-09-27: Skipping ZTF18abrwvkr, never reached limit 19.5
ZTF18aawrbuc
2018-09-27: Skipping ZTF18aawrbuc, never reached limit 19.5
ZTF18abporae
ZTF18abimrqx
2018-09-27: Skipping ZTF18abimrqx, never reached limit 19.5
ZTF18abaaeip
2018-09-27: Skipping ZTF18abaaeip, never reached limit 19.5
ZTF18abulqzb
ZTF18abnblhi
2018-09-27: Skipping ZTF18abnblhi, less than 5 dete

INFO:jnordin:2018-09-27: Skipping ZTF18abuyuqy, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abosrco, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abskuxt, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtphnh, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abtnwpa, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18absnmdd, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abuleom, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtnryk, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtoily, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18aapgbfm, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18aanxtko, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abdklhh, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18aanvpol, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abtkzfi, only 1 filters
IN

2018-09-27: Skipping ZTF18abuyuqy, never reached limit 19.5
ZTF18abosrco
2018-09-27: Skipping ZTF18abosrco, only 1 filters
ZTF18abskuxt
2018-09-27: Skipping ZTF18abskuxt, never reached limit 19.5
ZTF18abtphnh
2018-09-27: Skipping ZTF18abtphnh, less than 5 detections
ZTF18aaqzomm
ZTF18abtnwpa
2018-09-27: Skipping ZTF18abtnwpa, never reached limit 19.5
ZTF18abslbyb
ZTF18aamntui
ZTF18abbmenz
ZTF18abmjokl
ZTF18abtghbi
ZTF18abnvotk
ZTF18absnmdd
2018-09-27: Skipping ZTF18absnmdd, never reached limit 19.5
ZTF18abuleom
2018-09-27: Skipping ZTF18abuleom, never reached limit 19.5
ZTF18aaxudip
ZTF18abtnryk
2018-09-27: Skipping ZTF18abtnryk, never reached limit 19.5
ZTF18abtoily
2018-09-27: Skipping ZTF18abtoily, never reached limit 19.5
ZTF18aapgbfm
2018-09-27: Skipping ZTF18aapgbfm, never reached limit 19.5
ZTF18abujobx
ZTF18aanxtko
2018-09-27: Skipping ZTF18aanxtko, never reached limit 19.5
ZTF18ablprcf
ZTF18abdklhh
2018-09-27: Skipping ZTF18abdklhh, never reached limit 19.5
ZTF18aanvpol
2018-0

INFO:jnordin:2018-09-27: Skipping ZTF18abmmnqf, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtpgms, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abyzdoo, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abvvokw, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abvglon, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abxxviy, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abyxwrf, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18ablcwde, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abwgaek, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18absksjy, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abyvbat, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abuwqdi, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtrktm, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abxqzih, only 1 filters
INFO:jnordin

...

ZTF18abxzcky
2018-09-27: Skipping ZTF18abxzcky, less than 5 detections
ZTF18abyadtx
2018-09-27: Skipping ZTF18abyadtx, less than 5 detections
ZTF18abyzmsb
2018-09-27: Skipping ZTF18abyzmsb, less than 5 detections
ZTF18abxyudf
ZTF18abfsglv
ZTF18abwhsnx
2018-09-27: Skipping ZTF18abwhsnx, never reached limit 19.5
ZTF18abxxssh
2018-09-27: Skipping ZTF18abxxssh, less than 5 detections
ZTF18abxphmy
2018-09-27: Skipping ZTF18abxphmy, never reached limit 19.5
ZTF18abvkszn
2018-09-27: Skipping ZTF18abvkszn, never reached limit 19.5
ZTF18abxbmqh
2018-09-27: Skipping ZTF18abxbmqh, less than 5 detections
ZTF18abecggb
ZTF18abyaxyu
2018-09-27: Skipping ZTF18abyaxyu, never reached limit 19.5
ZTF18abyzeub
2018-09-27: Skipping ZTF18abyzeub, less than 5 detections
ZTF18abvfecb
ZTF18abzpwvn
2018-09-27: Skipping ZTF18abzpwvn, never reached limit 19.5
ZTF18abwugyc
2018-09-27: Skipping ZTF18abwugyc, less than 5 detections
ZTF18abyldss
2018-09-27: Skipping ZTF18abyldss, never reached limit 19.5
ZTF18abvosry


INFO:jnordin:2018-09-27: Skipping ZTF18abxmdan, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abwnciz, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abyfcqp, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18aaaeczy, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abxywlg, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abyikou, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abxzyss, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abvgizn, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abtwyej, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abxftqm, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abxwvnq, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abzppse, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abxavdf, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abvxoty, only 1 filter

2018-09-27: Skipping ZTF18abwkhjf, never reached limit 19.5
ZTF18abxmdan
2018-09-27: Skipping ZTF18abxmdan, never reached limit 19.5
ZTF18abwnciz
2018-09-27: Skipping ZTF18abwnciz, never reached limit 19.5
ZTF18abyfcqp
2018-09-27: Skipping ZTF18abyfcqp, less than 5 detections
ZTF18abwbtco
ZTF18aaaeczy
2018-09-27: Skipping ZTF18aaaeczy, never reached limit 19.5
ZTF18abxywlg
2018-09-27: Skipping ZTF18abxywlg, less than 5 detections
ZTF18abyikou
2018-09-27: Skipping ZTF18abyikou, never reached limit 19.5
ZTF18abxzyss
2018-09-27: Skipping ZTF18abxzyss, less than 5 detections
ZTF18abvgizn
2018-09-27: Skipping ZTF18abvgizn, never reached limit 19.5
ZTF18abtwyej
2018-09-27: Skipping ZTF18abtwyej, never reached limit 19.5
ZTF18abxftqm
2018-09-27: Skipping ZTF18abxftqm, less than 5 detections
ZTF18abxwvnq
2018-09-27: Skipping ZTF18abxwvnq, less than 5 detections
ZTF18abzacuc
ZTF18abzppse
2018-09-27: Skipping ZTF18abzppse, never reached limit 19.5
ZTF18abxavdf
2018-09-27: Skipping ZTF18abxavdf, 

INFO:jnordin:2018-09-27: Skipping ZTF18abtokcw, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abxznky, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF17aadhcyv, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abzqkui, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abztltu, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abxvwca, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abyckme, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abyekoi, never reached limit 19.5
INFO:jnordin:2018-09-27: Skipping ZTF18abujiho, only 1 filters
INFO:jnordin:2018-09-27: Skipping ZTF18abybuhw, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abvgjgb, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abcleke, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abywclh, less than 5 detections
INFO:jnordin:2018-09-27: Skipping ZTF18abflihq, never reached limit 19.5


2018-09-27: Skipping ZTF18abtokcw, only 1 filters
ZTF18abxznky
2018-09-27: Skipping ZTF18abxznky, less than 5 detections
ZTF17aadhcyv
2018-09-27: Skipping ZTF17aadhcyv, less than 5 detections
ZTF18abzqkui
2018-09-27: Skipping ZTF18abzqkui, never reached limit 19.5
ZTF18abydspi
ZTF18abztltu
2018-09-27: Skipping ZTF18abztltu, never reached limit 19.5
ZTF18abxvwca
2018-09-27: Skipping ZTF18abxvwca, less than 5 detections
ZTF18aazfthy
ZTF18abyckme
2018-09-27: Skipping ZTF18abyckme, less than 5 detections
ZTF18abyekoi
2018-09-27: Skipping ZTF18abyekoi, never reached limit 19.5
ZTF18abujiho
2018-09-27: Skipping ZTF18abujiho, only 1 filters
ZTF18abybuhw
2018-09-27: Skipping ZTF18abybuhw, less than 5 detections
ZTF18abyenql
ZTF18abvgjgb
2018-09-27: Skipping ZTF18abvgjgb, less than 5 detections
ZTF18abcleke
2018-09-27: Skipping ZTF18abcleke, less than 5 detections
ZTF18abywclh
2018-09-27: Skipping ZTF18abywclh, less than 5 detections
ZTF18abflihq
2018-09-27: Skipping ZTF18abflihq, never reached

In [10]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Quick plot of SN nbr k
    i : {0:undecided, 1:submit, 2:wait 3:bogus}
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = sncandidates[snnbr]
    if i == 'Wait':
        logger.info("Waiting for further obs for SN %s %s"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Submit':
        logger.info("Sending SN %s %s for possible follow-up"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'Garbage':
        logger.info("I never want to see SN %s %s again!"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(sncandidates):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    # Reset    
    snname = sncandidates[snnbr]
    

    phot = df_phot[ df_phot["ztf_name"]==snname ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']


    plt.title(snname + ' ' + str(snnbr+1) + ' out of '+str(len(sncandidates)))
    plt.plot(jd[filters==1]-Time.now().jd,magpsf[filters==1],'go')
    plt.plot(jd[filters==2]-Time.now().jd,magpsf[filters==2],'ro')
    plt.plot(jd[filters==3]-Time.now().jd,magpsf[filters==3],'ko')
    plt.gca().invert_yaxis()
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)
wiggy = widgets.RadioButtons(
    options=['Nothing','Wait', 'Submit', 'Garbage','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [11]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [12]:
# This is the all important list of decisions you have made
decisions = {}

In [ ]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

In [ ]:
# Save the full decisions to the log
logger.info("Full decision set: %s"%(decisions))

In [13]:
# Now fun starts for real. Lets download all the candidates and check whether we find all the candidate ids there
# (cause are the only that are useful for saving). This will take a while.
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 192 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-09-28 11:51:13 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration number 0: 43 jobs to do
INFO:marshaltools.ProgramList:jobs are done: total 43, failed: 0
INFO:marshaltools.ProgramList:Fetched 3263 candidates in 1.57e+01 sec


In [14]:
ingestoldcand_start = Time.now()
toingest = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    logger.info("Trying to ingest %s through avroid %s"%(sncandidates[snnbr],avroid))
    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        logger.info('... already a candidate with id %s'%(canddate['candid']))
        continue
    toingest.append(avroid)

toingest = [634445152015015010, 633163410315015008, 634211893715015190, 634306734915015002, 634274383415015037, 634182006115015005, 634523275415015006, 634418762815015009, 634226450415015028, 634356423615015001]
    
print("Starting ingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_start))
failed = pl.ingest_avro(toingest)
logger.info("Tried to ingest %s, failed for %s"%(toingest,failed))
ingestoldcand_end = Time.now()
print("Stoppedingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_end))

INFO:marshaltools.ProgramList:Trying to ingest 10 candidate(s) to marshal program AMPEL Test (ingest ID 4)


Starting ingesting at 2018-09-28 11:52:04.913613 (useful if you want to check scanning page directly)


INFO:marshaltools.ProgramList:Attempt 1: done ingesting candidates.
INFO:marshaltools.ProgramList:veryfying ingestion looking at candidates ingested between 2018-09-28 11:52:04.916 and 2018-09-28 11:52:33.736
INFO:marshaltools.ProgramList:attempt # 1. Of the desired candidates 7 successfully ingested, 3 failed
INFO:marshaltools.ProgramList:Attempt 2: done ingesting candidates.
INFO:marshaltools.ProgramList:veryfying ingestion looking at candidates ingested between 2018-09-28 11:52:04.916 and 2018-09-28 11:52:44.740
INFO:marshaltools.ProgramList:attempt # 2. Of the desired candidates 0 successfully ingested, 3 failed
INFO:marshaltools.ProgramList:Attempt 3: done ingesting candidates.
INFO:marshaltools.ProgramList:veryfying ingestion looking at candidates ingested between 2018-09-28 11:52:04.916 and 2018-09-28 11:52:56.041
INFO:marshaltools.ProgramList:attempt # 3. Of the desired candidates 0 successfully ingested, 3 failed
INFO:jnordin:Tried to ingest [634445152015015010, 63316341031501

Stoppedingesting at 2018-09-28 11:52:58.220543 (useful if you want to check scanning page directly)


In [15]:
# We do this again because it is soo funny
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 192 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-09-28 11:53:39 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration number 0: 43 jobs to do
INFO:marshaltools.ProgramList:jobs are done: total 43, failed: 0
INFO:marshaltools.ProgramList:Fetched 3263 candidates in 1.61e+01 sec


In [ ]:
# And we do this again as well. If this worked we should now have all candidates saved
togrowth = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        continue
    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    msg = "Failed to ingest alert %s for SN %s"%(avroid,sncandidates[snnbr])
    logger.info(msg)
    togrowth.append(msg)
    


In [ ]:
# Write an email to Mansi/Growth team with the following content
print(togrowth)
# And when we hear back from them this SN should be saved. I hope they do it for us

In [ ]:
# Ok - this does not work since we have the wrong candidate id...
missing_candidsne = []
# Now it is time to save remaining candidates
savecand_start = Time.now()
print("Starting saving at %s (not really that useful)"%(savecand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    try:
        candid = soupids[sncandidates[snnbr]]
        logger.info("Try to save %s through candid %s"%(sncandidates[snnbr],candid))
        pl.save_source(candid,42)
    except KeyError:
        logger.info("Lacking candid for %s"%(sncandidates[snnbr]))
        missing_candidsne.append(snnbr)


        
savecand_end = Time.now()
print("Stopped saving at %s"%(savecand_end))

In [ ]:
# Check whether we have missing candidate ids
print("Missing candidate id for these sne: %s"%(missing_candidsne))

In [25]:
# Reload saved sources
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 200 saved sources for program AMPEL Test.


In [ ]:
# Loop through them again to check they are all saved
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=False)
    if canddate is None:
        logger.info('SN %s was not saved. Oj oj.'%(sncandidates[snnbr]))
        continue

    logger.info('SN %s was saved'%(sncandidates[snnbr]))


In [ ]:
## Hopefully done. If all of the candidates were saved (no ojojs) you are done with scanning!
# If candidate IDs are still missing you have to try to scrape the webpages for them as below. Have fun!

In [3]:
# Trying to get candid through the seargant
from ampel.pipeline.t3.sergeant import marshal_functions
tbase = Time.now()
allcandidates = []

In [4]:
for k in range(3):
    print(k)
    tend = Time(tbase.jd-k,format='jd')
    tstart = Time(tbase.jd-k-1,format='jd')
    ser = marshal_functions.Sergeant(program_name='AMPEL Test',start_date=tstart.iso,end_date=tend.iso, marshalusr='jnordin',marshalpwd='oskarthebaseplayer')
    ser.cutprogramidx = 42
    cands = ser.list_scan_sources()
    print("Found %s cands"%(len(cands)))
    allcandidates.extend(cands)

0
Found 40 cands
1
Found 39 cands
2
Found 24 cands


In [7]:
for cand in allcandidates:
    if cand["name"] in soupids.keys():
        continue
    print("new candidate %s"%(cand["name"]))
    soupids[cand["name"]] = cand["candid"]

new candidate ZTF18abufaej
new candidate ZTF18abvrzqj
new candidate ZTF18abvxqtp
new candidate ZTF18abmfmos
new candidate ZTF18abvfecb
new candidate ZTF18abwbtco
new candidate ZTF18abulqzb
new candidate ZTF18ablprcf
new candidate ZTF18abmwfxd
new candidate ZTF18abxdkni
new candidate ZTF18absgvqb
new candidate ZTF18abporae
new candidate ZTF18ablqjws
new candidate ZTF18abmrhkl
new candidate ZTF18abmoqpm
new candidate ZTF18abysimb
new candidate ZTF18abxbmcl
new candidate ZTF18abytsmi
new candidate ZTF18abzotbm
new candidate ZTF18abyymsl
new candidate ZTF18abnoedw
new candidate ZTF18abwpddc
new candidate ZTF18abxqdlm
new candidate ZTF18abwwfyo
new candidate ZTF18abxbwwf
new candidate ZTF18ablrlbm
new candidate ZTF18abymobs
new candidate ZTF18abylttk
new candidate ZTF18abymonp
new candidate ZTF18aatycgd
new candidate ZTF18abywfme
new candidate ZTF18abywgcs
new candidate ZTF18abrwqze
new candidate ZTF18abcftam


In [8]:
# Save the added dict of candidate id
with open('sncandid_soup.json', 'w') as fp:
    json.dump(soupids, fp)

In [10]:
foo = """ZTF18ablrndb 
ZTF18aawlhkh
ZTF18abporae
ZTF17aabihdn
ZTF18aawlhkh
ZTF18absgvqb
ZTF18ablprcf
ZTF18abufaej
ZTF18absrcps
ZTF18abvfecb
ZTF18abwbtco
ZTF18abxdkni
ZTF18abvrzqj"""

In [11]:
foo2 = foo.split("\n")

In [12]:
print(foo2)

['ZTF18ablrndb ', 'ZTF18aawlhkh', 'ZTF18abporae', 'ZTF17aabihdn', 'ZTF18aawlhkh', 'ZTF18absgvqb', 'ZTF18ablprcf', 'ZTF18abufaej', 'ZTF18absrcps', 'ZTF18abvfecb', 'ZTF18abwbtco', 'ZTF18abxdkni', 'ZTF18abvrzqj']


In [ ]:
# cheat for 09 23


In [23]:
for snname in foo2:

    try:
        candid = soupids[snname]
        logger.info("Try to save %s through candid %s"%(snname,candid))
        pl.save_source(candid,42)
    except KeyError:
        logger.info("Lacking candid for %s"%(snname))


INFO:jnordin:Lacking candid for ZTF18ablrndb 
INFO:jnordin:Try to save ZTF18aawlhkh through candid 1545871
INFO:marshaltools.ProgramList:Saving source 1545871 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18abporae through candid 1554627
INFO:marshaltools.ProgramList:Saving source 1554627 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF17aabihdn through candid 1546760
INFO:marshaltools.ProgramList:Saving source 1546760 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18aawlhkh through candid 1545871
INFO:marshaltools.ProgramList:Saving source 1545871 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18absgvqb through candid 1559399
INFO:marshaltools.ProgramList:Saving source 1559399 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnor

In [26]:
for snname in foo2:
    canddate = pl.find_source(snname,include_candidates=False)
    if canddate is None:
        logger.info('SN %s was not saved. Oj oj.'%(snname))
        continue

    logger.info('SN %s was saved'%(snname))

INFO:jnordin:SN ZTF18ablrndb  was not saved. Oj oj.
INFO:jnordin:SN ZTF18aawlhkh was not saved. Oj oj.
INFO:jnordin:SN ZTF18abporae was saved
INFO:jnordin:SN ZTF17aabihdn was not saved. Oj oj.
INFO:jnordin:SN ZTF18aawlhkh was not saved. Oj oj.
INFO:jnordin:SN ZTF18absgvqb was saved
INFO:jnordin:SN ZTF18ablprcf was saved
INFO:jnordin:SN ZTF18abufaej was saved
INFO:jnordin:SN ZTF18absrcps was not saved. Oj oj.
INFO:jnordin:SN ZTF18abvfecb was saved
INFO:jnordin:SN ZTF18abwbtco was saved
INFO:jnordin:SN ZTF18abxdkni was saved
INFO:jnordin:SN ZTF18abvrzqj was saved


In [28]:
for snname in foo2:
    try:
        print('%s %s'%(snname,soupids[snname]))
    except KeyError:
        print("noooo.. %s"%(snname))

noooo.. ZTF18ablrndb 
ZTF18aawlhkh 1545871
ZTF18abporae 1554627
ZTF17aabihdn 1546760
ZTF18aawlhkh 1545871
ZTF18absgvqb 1559399
ZTF18ablprcf 1555641
ZTF18abufaej 1559090
ZTF18absrcps 1535950
ZTF18abvfecb 1556357
ZTF18abwbtco 1556215
ZTF18abxdkni 1554861
ZTF18abvrzqj 1558952
